In [1]:
import importlib
from tqdm import tqdm
import json
calculate_doc_term_recall = importlib.import_module('train-deep-ct').calculate_doc_term_recall
calculate_qtokens = importlib.import_module('train-deep-ct').calculate_qtokens

!nosetests
from util import clean_qtokens

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [2]:
DIR = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/deep-ct/'

try:
    anchorText = clean_qtokens(json.load(open(DIR + 'cc-2019-47-deepct-qtokens.json', 'r')))
except Exception as e:
    print(e)
    print('Comment the following lines to create cc-2019-47-anchors.jsonl.')
    #anchorText = {}
    #with open(DIR + 'cc-2019-47-anchors.jsonl') as f:
        #for l in tqdm(f):
            #l = json.loads(l)
            #anchorText[l['targetDocumentId']] = calculate_qtokens(l['anchorText'])
        #json.dump(anchorText, open(DIR + 'cc-2019-47-deepct-qtokens.json', 'w'))

From qtokens with 1163273 documents, I filtered terms that may cause train/test overlap so that I now have 924621 documents.


In [8]:
!ls -lha {DIR}cc-2019-47-deepct-qtokens.json

-rw-rw-r-- 1 root root 346M Sep 10 06:56 /mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/deep-ct/cc-2019-47-deepct-qtokens.json


In [3]:
anchorText['D2030531']

{'hotel': 0.8333333333333333,
 'islay': 0.3333333333333333,
 'hostel': 0.3333333333333333,
 'com': 0.16666666666666666,
 'http': 0.16666666666666666,
 'www': 0.16666666666666666,
 'html': 0.16666666666666666,
 'islayinfo': 0.16666666666666666}

In [4]:
len(anchorText)

924621

In [5]:
with open(DIR + 'msmarco-document-passages.jsonl') as docs, open(DIR + 'deep-ct-training-data-cc-2019-47-test-overlap-removed.jsonl', 'w') as out_file:
    for doc in tqdm(docs):
        doc = json.loads(doc)
        doc_id = doc['id']
        if doc_id not in anchorText:
            continue
            
        for passage_id, passage in doc['passages']:
            for i in calculate_doc_term_recall([{"doc":{"title": passage}}], anchorText[doc_id]):
                i['doc']['position'] = passage_id
                i['doc']['id'] = doc_id + '___' + str(passage_id)
                out_file.write(json.dumps(i) + '\n')

3213835it [4:36:22, 193.81it/s]


In [6]:
saved_docs = []
examples = 0

with open(DIR + 'deep-ct-training-data-cc-2019-47-test-overlap-removed.jsonl', 'r') as docs:
    for l in tqdm(docs):
        l = json.loads(l)
        examples += 1
        saved_docs += [l['doc']['id'].split('___')[0]]

print('Examples: ', examples)
print('Covered Documents: ', len(set(saved_docs)))

9412482it [01:35, 98607.53it/s] 


Examples:  9412482
Covered Documents:  924621


In [7]:
docs_to_passage_count = {}

with open(DIR + 'deep-ct-training-data-cc-2019-47-test-overlap-removed.jsonl', 'r') as docs:
    for l in tqdm(docs):
        l = json.loads(l)
        doc_id = l['doc']['id'].split('___')[0]
        if doc_id not in docs_to_passage_count:
            docs_to_passage_count[doc_id] = 0
        docs_to_passage_count[doc_id] = docs_to_passage_count[doc_id] + 1


9412482it [01:41, 92632.14it/s]


In [10]:
from random import shuffle
selected_ids = []
count = 0
doc_ids = [i for i in docs_to_passage_count.keys()]
shuffle(doc_ids)

for doc_id in doc_ids:
    count += docs_to_passage_count[doc_id]
    selected_ids += [doc_id]
    if count > 2079858:
        break

selected_ids = set(selected_ids)
print('Have ', len(selected_ids), ' Ids')
print('Have ', count, ' Examples')

Have  203494  Ids
Have  2079888  Examples


In [11]:
with open(DIR + 'deep-ct-training-data-cc-2019-47-test-overlap-removed.jsonl', 'r') as docs, open(DIR + 'deep-ct-training-data-cc-2019-47-sampled-test-overlap-removed.jsonl', 'w') as out_file:
    for l in tqdm(docs):
        doc_id = json.loads(l)['doc']['id'].split('___')[0]
        if doc_id in selected_ids:
            out_file.write(l)

9412482it [01:42, 91779.46it/s] 


In [14]:
!cat {DIR}deep-ct-training-data-cc-2019-47-sampled-test-overlap-removed.jsonl|grep 'D15500___2' | jq '.'

{
  "doc": {
    "title": "Edit Article\nHow to Learn Martial Arts \"Pressure Points\"12 Parts: Understanding pressure points Head Neck Shoulder Throat Jaw zone\nForearm/crevice Hands Torso region Feet Possible therapeutic uses\nPracticing the pressure points Community Q&AThe traditional definition of a pressure point is a point that, when pressure is applied, produces crippling pain. \nThis is learnt in a Chinese martial art called Dim Mak based on acupuncture pressure points, but this art is very restricted and needs and understanding of Chinese acupuncture points. \nBecause of this, this article can only provide information on on fragile areas that we'll call vulnerable points. \nThis is used to exploit a weakness or vulnerability in the human body to gain an advantage over an opponent. \nWhen using these pressure points one must be particularly careful as it is easy to kill someone accidentally, such as a friend or even an enemy. \nAt that point, you enter the legal system, which g

In [15]:
!cat {DIR}deep-ct-training-data-cc-2019-47-test-overlap-removed.jsonl|grep 'D15500___2' | jq '.'

{
  "doc": {
    "title": "Edit Article\nHow to Learn Martial Arts \"Pressure Points\"12 Parts: Understanding pressure points Head Neck Shoulder Throat Jaw zone\nForearm/crevice Hands Torso region Feet Possible therapeutic uses\nPracticing the pressure points Community Q&AThe traditional definition of a pressure point is a point that, when pressure is applied, produces crippling pain. \nThis is learnt in a Chinese martial art called Dim Mak based on acupuncture pressure points, but this art is very restricted and needs and understanding of Chinese acupuncture points. \nBecause of this, this article can only provide information on on fragile areas that we'll call vulnerable points. \nThis is used to exploit a weakness or vulnerability in the human body to gain an advantage over an opponent. \nWhen using these pressure points one must be particularly careful as it is easy to kill someone accidentally, such as a friend or even an enemy. \nAt that point, you enter the legal system, which g